# Extracting SNP-aware and reference genome sequences for MPRA data

#### In this notebook, we adapt our SNP pre-procesing pipeline to extract sequences from the human genome. The Massively Parallel Report Assay (MPRA) data come from https://www.biorxiv.org/content/10.1101/2023.03.05.531189v1, which measures the regulatory activity of sequences through a reporter system. We first mapped those sequences to the reference genome through sequence alignment. Then based on the coordinates from the alignment, we retrieve the sequence with SNP encoded and flanking regions.

In [1]:
import numpy as np
import pandas as pd
from scipy import sparse

from bmfm_targets.datasets.SNPdb.tabix_converter import sample_variant

np.set_printoptions(precision=3,suppress=True)

/dccstor/fmm/users/hongyang/anaconda3/envs/bmfm-targets/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### New functions to extract sequences based on coordinates and calculate the percentage of variations

In [2]:
def extract_subsequence(sequence, left, right, len_total=None):
    if len_total:
        center = (left + right) // 2
        left = center - len_total // 2
        right = left + len_total
    return sequence[left:right]

def count_variation_percent(sequence):
    return sum([x not in 'ACGTN' for x in sequence]) / len(sequence)

In [3]:
path0 = '/dccstor/bmfm-targets/data/omics/genome/snpdb/raw/matrices/'
SNPDB_RESOURCES_PATH = "/dccstor/bmfm-targets/data/omics/genome/snpdb/raw/resources/"
nucleotide_lexicon_path = SNPDB_RESOURCES_PATH + "nucleotide_lexicon.json"
biallele_lexicon_path = SNPDB_RESOURCES_PATH + "biallele_lexicon.json"

input_file = './data/K562_clean_outfmt6processed.out'
output1_path = './data/K562_biallele_sequence_200.txt'
output2_path = './data/K562_biallele_sequence_20kb.txt'

#### There are special "chromosomes" to be excluded

In [4]:
df0 = pd.read_csv(input_file,sep='\t')
df0.chr.unique()

array(['1', '10', 'hs37d5', '11', 'GL000191.1', 'GL000202.1', '12',
       'GL000223.1', 'GL000226.1', 'GL000212.1', '13', '14', '15', '16',
       '17', 'GL000204.1', '18', '19', 'GL000209.1', '2', '20',
       'GL000198.1', '9', '21', 'GL000205.1', 'GL000193.1', 'GL000210.1',
       'GL000241.1', '22', 'GL000246.1', 'GL000242.1', 'GL000249.1',
       'GL000248.1', '3', 'GL000192.1', 'GL000222.1', '4', '5',
       'GL000227.1', '6', '7', '8', 'GL000197.1', 'GL000196.1',
       'GL000200.1', 'GL000211.1', 'GL000201.1', 'X', 'Y', 'GL000219.1'],
      dtype=object)

#### Summary of the mapped sequences in each chromosome

In [5]:
df0.chr.value_counts()

chr
1             22372
2             19182
11            18154
6             13433
8             12474
3             11968
7             11130
16            10749
X              9769
12             9668
20             9350
5              9141
17             9123
10             9106
4              8569
19             8127
9              7554
15             6343
14             5237
22             4258
18             3438
13             3415
21             2670
hs37d5          500
Y                56
GL000193.1       16
GL000192.1       13
GL000191.1       10
GL000223.1       10
GL000202.1        8
GL000204.1        8
GL000249.1        8
GL000197.1        6
GL000226.1        6
GL000205.1        6
GL000227.1        6
GL000201.1        6
GL000209.1        6
GL000211.1        4
GL000198.1        4
GL000210.1        4
GL000242.1        4
GL000248.1        4
GL000222.1        4
GL000212.1        2
GL000246.1        2
GL000241.1        2
GL000196.1        2
GL000200.1        2
GL000219.1      

#### Extract mapped sequences with/without flank (length = ~200 or 20kb)

In [6]:
id_all = []
subsequence1_all = []
subsequence2_all = []
for i in range(1,23):
    target_chr = 'chr' + str(i)
    the_df = df0[df0.chr == str(i)]
    print('chr' + str(i), len(the_df))
    snp_probability_matrix = sparse.load_npz(path0 + 'snp_prob_' + target_chr + '.npz')
    ## encoded sequence for a chromosome
    encoded_seq = sample_variant(
        snp_probability_matrix,
        nucleotide_lexicon_path,
        biallele_lexicon_path,
        replacement=False,
    )
    for j in range(len(the_df)):
        id_all.append(the_df['id'].iloc[j])
        left =  min(the_df['s_start'].iloc[j], the_df['s_end'].iloc[j])
        right =  max(the_df['s_start'].iloc[j], the_df['s_end'].iloc[j]) + 1
        subsequence1_all.append(extract_subsequence(encoded_seq, left, right))
        subsequence2_all.append(extract_subsequence(encoded_seq, left, right, len_total=20000))

chr1 22372


Sampling from row 0


Sampling from row 1000000


Sampling from row 2000000


Sampling from row 3000000


...

Sampling from row 47000000


Sampling from row 48000000


Sampling from row 49000000


Sampling from row 50000000


Sampling from row 51000000


In [7]:
with open(output1_path, "w") as f:
    for the_id, subsequence in zip(id_all,subsequence1_all):
        f.write(f'{the_id},{subsequence}\n')
with open(output2_path, "w") as f:
    for the_id, subsequence in zip(id_all,subsequence2_all):
        f.write(f'{the_id},{subsequence}\n')

#### Calculate percentage of variations for the original ~200bp sequeneces and longer 20kb sequences

In [8]:
var_percent1 = [count_variation_percent(x) for x in subsequence1_all]
print("percentage of variation(200): %.2f%%" % (sum(var_percent1)/len(var_percent1)*100))

percentage of variation(200): 0.75%


In [9]:
var_percent2 = [count_variation_percent(x) for x in subsequence2_all]
print("percentage of variation(20kb): %.2f%%" % (sum(var_percent2)/len(var_percent2)*100))

percentage of variation(20kb): 0.94%
